<a href="https://colab.research.google.com/github/Suhail372/files_for_chatbot/blob/master/chatbot_streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Suhail372/files_for_chatbot

Cloning into 'files_for_chatbot'...
remote: Enumerating objects: 217, done.
remote: Counting objects: 100% (217/217), done.
remote: Compressing objects: 100% (102/102), done.
remote: Total 217 (delta 144), reused 170 (delta 112), pack-reused 0 (from 0)
Receiving objects: 100% (217/217), 18.31 MiB | 9.54 MiB/s, done.
Resolving deltas: 100% (144/144), done.


In [8]:
import os
os.chdir('/content/files_for_chatbot')

In [2]:
!pip install sentence-transformers faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.4 MB/s eta 0:00:00


In [3]:
!pip install streamlit pyngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.6 MB/s eta 0:00:00


In [9]:
from pyngrok import ngrok

# Start ngrok to tunnel the Streamlit app
ngrok.set_auth_token('2nCn1vEMlxC7W9zdbtOGVwB8DR8_2yTPhCEvMraV4WdgpNvWo')  # Optional
public_url = ngrok.connect("http://localhost:8501")  # Use localhost
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:8501\"")




 * ngrok tunnel "NgrokTunnel: "https://2bf8-34-82-230-42.ngrok-free.app" -> "http://localhost:8501"" -> "http://127.0.0.1:8501"


In [10]:
# Start the Streamlit app
!streamlit run /content/files_for_chatbot/copy_of_chatbot.py &




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.82.230.42:8501

2024-10-10 03:52:27.540828: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-10 03:52:27.565227: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-10 03:52:27.572441: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-10 03:52:27.589233: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To 